In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import torchvision

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.utils import save_image

import pandas as pd
import numpy as np
import scipy.io
import skimage.io

from PIL import Image, ImageFilter

# Define Model

In [2]:
class Encoder(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(out_channel)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x)
        x, idx = F.max_pool2d(x, kernel_size=2, stride=2, return_indices=True)
        return x, idx

In [3]:
class Decoder(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(out_channel)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = F.relu(x)
        return x

In [4]:
class SegNetBasic(nn.Module):
    """ 
        SegNet Basic is a smaller version of SegNet
        Please refer to this repository:
        https://github.com/0bserver07/Keras-SegNet-Basic
    """
    
    def __init__(self, in_channel, out_channel):
        super().__init__()
        
        self.encoder1 = Encoder(in_channel, 64)
        self.encoder2 = Encoder(64, 80)
        self.encoder3 = Encoder(80, 96)
        self.encoder4 = Encoder(96, 128)
        
        self.decoder1 = Decoder(128, 96)
        self.decoder2 = Decoder(96, 80)
        self.decoder3 = Decoder(80, 64)
        self.decoder4 = Decoder(64, out_channel)
        
    def forward(self, x):
        size1 = x.size()
        x, idx1 = self.encoder1(x)

        size2 = x.size()
        x, idx2 = self.encoder2(x)

        size3 = x.size()
        x, idx3 = self.encoder3(x)
        
        size4 = x.size()
        x, idx4 = self.encoder4(x)

        x = F.max_unpool2d(x, idx4, kernel_size=2, stride=2, output_size=size4)
        x = self.decoder1(x)
        
        x = F.max_unpool2d(x, idx3, kernel_size=2, stride=2, output_size=size3)
        x = self.decoder2(x)

        x = F.max_unpool2d(x, idx2, kernel_size=2, stride=2, output_size=size2)
        x = self.decoder3(x)

        x = F.max_unpool2d(x, idx1, kernel_size=2, stride=2, output_size=size1)
        x = self.decoder4(x)

        return x

# Dataset

In [5]:
# count the number of pixels in each affordance
import glob

path_list = glob.glob('./part-affordance-dataset/tools/*', recursive=True)
print(path_list)
print(len(path_list))

['./part-affordance-dataset/tools/mug_15', './part-affordance-dataset/tools/knife_12', './part-affordance-dataset/tools/spoon_01', './part-affordance-dataset/tools/ladle_04', './part-affordance-dataset/tools/trowel_02', './part-affordance-dataset/tools/scissors_01', './part-affordance-dataset/tools/bowl_08', './part-affordance-dataset/tools/tenderizer_02', './part-affordance-dataset/tools/shovel_02', './part-affordance-dataset/tools/bowl_07', './part-affordance-dataset/tools/spoon_02', './part-affordance-dataset/tools/saw_03', './part-affordance-dataset/tools/cup_04', './part-affordance-dataset/tools/spoon_06', './part-affordance-dataset/tools/spoon_04', './part-affordance-dataset/tools/spoon_08', './part-affordance-dataset/tools/mug_19', './part-affordance-dataset/tools/pot_01', './part-affordance-dataset/tools/knife_11', './part-affordance-dataset/tools/scoop_02', './part-affordance-dataset/tools/bowl_02', './part-affordance-dataset/tools/cup_05', './part-affordance-dataset/tools/mug

In [6]:
image_path_list = []

for path in path_list:
    i = glob.glob(path + '/*rgb.jpg')
    
    image_path_list += i

In [7]:
class_path_list = []

for path in image_path_list:
    c = path[:-7] + 'label.mat'
    class_path_list.append(c)

In [11]:
df = pd.DataFrame({
    'image_path': image_path_list,
    'class_path': class_path_list},
    columns=['image_path', 'class_path']
)

In [5]:
# # write data as csv_file
# df.to_csv('image_class_path.csv', index=None)

In [14]:
df = pd.read_csv("image_class_path.csv")

In [15]:
df_s = df.sample(frac=1)

In [16]:
df_s

,image_path,class_path
24546,./part-affordance-dataset/tools/hammer_04/hamm...,./part-affordance-dataset/tools/hammer_04/hamm...
25558,./part-affordance-dataset/tools/mug_04/mug_04_...,./part-affordance-dataset/tools/mug_04/mug_04_...
17133,./part-affordance-dataset/tools/bowl_05/bowl_0...,./part-affordance-dataset/tools/bowl_05/bowl_0...
26672,./part-affordance-dataset/tools/spoon_03/spoon...,./part-affordance-dataset/tools/spoon_03/spoon...
10798,./part-affordance-dataset/tools/mallet_03/mall...,./part-affordance-dataset/tools/mallet_03/mall...
11685,./part-affordance-dataset/tools/mug_14/mug_14_...,./part-affordance-dataset/tools/mug_14/mug_14_...
5518,./part-affordance-dataset/tools/scoop_02/scoop...,./part-affordance-dataset/tools/scoop_02/scoop...
7302,./part-affordance-dataset/tools/bowl_03/bowl_0...,./part-affordance-dataset/tools/bowl_03/bowl_0...
28594,./part-affordance-dataset/tools/scissors_04/sc...,./part-affordance-dataset/tools/scissors_04/sc...
26333,./part-affordance-dataset/tools/mug_02/mug_02_...,./part-affordance-dataset/tools/mug_02/mug_02_...


In [17]:
28843*0.8

23074.4

In [43]:
df_train = df_s[:23100]

In [19]:
df_train.to_csv('train.csv', index=None)

In [20]:
df_test = df_s[23100:]

In [21]:
df_test.to_csv('test.csv', index=None)

### the number of the pixels before center crop

In [172]:
# cnt_dict = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0}

# for path in class_path_list:
#     mat = scipy.io.loadmat(path)
#     num, cnt = np.unique(mat['gt_label'], return_counts=True)
    
#     for n, c in zip(num, cnt):
#         cnt_dict[n] += c

cnt_dict

0: 8723340880,<br>
 1: 34159284,<br>
 2: 16009552,<br>
 3: 12433420,<br>
 4: 38476964,<br>
 5: 6773540,<br>
 6: 9273880,<br>
 7: 20102080<br>

# Define Dataset Class

In [5]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import pandas as pd
import numpy as np
import scipy.io
import skimage.io
from PIL import Image, ImageFilter
from sklearn.model_selection import train_test_split

In [6]:
class PartAffordanceDataset(Dataset):
    """Part Affordance Dataset"""
    
    def __init__(self, csv_file, transform=None):
        super().__init__()
        
        self.image_class_path = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.image_class_path)
    
    def __getitem__(self, idx):
        image_path = self.image_class_path.iloc[idx, 0]
        class_path = self.image_class_path.iloc[idx, 1]
        image = skimage.io.imread(image_path) # read as numpy array
        cls = scipy.io.loadmat(class_path)["gt_label"]
        
        sample = {'image': image, 'class': cls}
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample

In [7]:
def crop_center_numpy(array, crop_height, crop_weight):
    h, w = array.shape
    return array[h//2 - crop_height//2: h//2 + crop_height//2,
                 w//2 - crop_weight//2: w//2 + crop_weight//2
                ]

In [8]:
def crop_center_pil_image(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

In [9]:
class CenterCrop(object):
    def __call__(self, sample):
        image, cls = sample['image'], sample['class']
        
        image = Image.fromarray(np.uint8(image))
        
        image = crop_center_pil_image(image, 320, 240)
        cls = crop_center_numpy(cls, 240, 320)
        
        image = np.asarray(image)
        
        return {'image': image, 'class': cls}

In [10]:
class ToTensor(object):
    def __call__(self, sample):
        image, cls = sample['image'], sample['class']
        
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(image).float(), 
                'class': torch.from_numpy(cls).long()}

In [11]:
# data = PartAffordanceDataset('image_class_path.csv',
#                                 transform=transforms.Compose([
#                                     CenterCrop(),
#                                     ToTensor()
#                                 ]))

# data_loader = DataLoader(data, batch_size=10, shuffle=False)

In [12]:
# mean = 0
# std = 0
# n = 0

# for sample in data_loader:
#     img = sample['image']   
#     img = img.view(len(img), 3, -1)
#     mean += img.mean(2).sum(0)
#     std += img.std(2).sum(0)
#     n += len(img)
    
# mean /= n
# std /= n

In [13]:
mean=[55.8630, 59.9099, 91.7419]
std=[31.6852, 29.8496, 19.0835]

In [14]:
class Normalize(object):
    def __call__(self, sample):
        image, cls = sample['image'], sample['class']
        
        image = transforms.functional.normalize(image, mean, std)
        
        return {'image': image, 'class': cls}

In [15]:
train_data = PartAffordanceDataset('train.csv',
                                transform=transforms.Compose([
                                    CenterCrop(),
                                    ToTensor(),
                                    Normalize()
                                ]))

In [16]:
test_data = PartAffordanceDataset('test.csv',
                                transform=transforms.Compose([
                                    CenterCrop(),
                                    ToTensor(),
                                    Normalize()
                                ]))

In [34]:
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
test_loader = DataLoader(test_data, batch_size=10, shuffle=False)
pred_loader = DataLoader(test_data, batch_size=10, shuffle=True)

### count the number of pixels in each class after center crop

In [35]:
# dataset = PartAffordanceDataset('image_class_path.csv',
#                                 transform=transforms.Compose([
#                                     CenterCrop(),
#                                     ToTensor()
#                                 ]))
# data_laoder = DataLoader(dataset, batch_size=100, shuffle=False)

In [36]:
cnt_dict = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0}

for sample in data_laoder:
    img = sample['class'].numpy()
    
    num, cnt = np.unique(img, return_counts=True)
    
    for n, c in zip(num, cnt):
        cnt_dict[n] += c

In [37]:
cnt_dict

{0: 2078085712,
 1: 34078992,
 2: 15921090,
 3: 12433420,
 4: 38473752,
 5: 6773528,
 6: 9273826,
 7: 20102080}

cnt_dict

0: 2078085712,  
 1: 34078992,  
 2: 15921090,  
 3: 12433420,  
 4: 38473752,  
 5: 6773528,  
 6: 9273826,  
 7: 20102080  

# Training

### class weight

In [17]:
class_num = torch.tensor([2078085712, 34078992, 15921090, 12433420, 
                          38473752, 6773528, 9273826, 20102080])

total = class_num.sum().item()
print(total)

2215142400


In [18]:
class_num[0].float() / (100.0 * class_num.float())

tensor([0.0100, 0.6098, 1.3052, 1.6714, 0.5401, 3.0680, 2.2408, 1.0338])

In [19]:
frequency = class_num.float() / total
median = torch.median(frequency)

In [20]:
class_weight = median / frequency

In [21]:
class_weight

tensor([0.0077, 0.4672, 1.0000, 1.2805, 0.4138, 2.3505, 1.7168, 0.7920])

In [22]:
from tensorboardX import SummaryWriter
import tqdm

In [40]:
def eval_model(model, test_loader, device='cpu'):
    model.eval()
    
    intersection = torch.zeros(8)   # the dataset has 8 classes including background
    union = torch.zeros(8)
    
    for sample in test_loader:
        x, y = sample['image'], sample['class']
        
        x = x.to(device)
        y = y.to(device)
        
        with torch.no_grad():
            _, ypred = model(x).max(1)    # y_pred.shape => (N, 240, 320)
        
        for i in range(8):
            y_i = (y == i)           
            ypred_i = (ypred == i)   
            
            inter = (y_i.byte() & ypred_i.byte()).float().sum().to('cpu')
            intersection[i] += inter
            union[i] += (y_i.float().sum() + ypred_i.float().sum()).to('cpu') - inter
    
    """ iou[i] is the IoU of class i """
    iou = intersection / union
    
    return iou

In [41]:
def train_model(model, train_loader, test_loader, optimizer_cls=optim.Adam, 
                criterion=nn.CrossEntropyLoss(), max_epoch=200, device='cpu', writer=None):
    
    model.to(device)
    
    train_losses = []
    val_iou = []
    mean_iou = []
    best_iou = 0.0
    
    optimizer = optimizer_cls(model.parameters(), lr=0.01)
    
    for epoch in range(max_epoch):
        model.train()
        running_loss = 0.0
        
        for i, sample in tqdm.tqdm(enumerate(train_loader), total=len(train_loader)):
            optimizer.zero_grad()
            
            x, y = sample['image'], sample['class']
            
            x = x.to(device)
            y = y.to(device)

            h = model(x)
            loss = criterion(h, y)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        train_losses.append(running_loss / i)
        
        val_iou.append(eval_model(model, test_loader, device))
        mean_iou.append(val_iou[-1].mean().item())
        
        if best_iou < mean_iou[-1]:
            best_iou = mean_iou[-1]
            torch.save(model.state_dict(), "./SegNet_with_class_weight(median)_results/best_iou_model.prm")
        
        if writer is not None:
            writer.add_scalar("train_loss", train_losses[-1], epoch)
            writer.add_scalar("mean_IoU", mean_iou[-1], epoch)
            writer.add_scalars("class_IoU", {'iou of class 0': val_iou[-1][0],
                                           'iou of class 1': val_iou[-1][1],
                                           'iou of class 2': val_iou[-1][2],
                                           'iou of class 3': val_iou[-1][3],
                                           'iou of class 4': val_iou[-1][4],
                                           'iou of class 5': val_iou[-1][5],
                                           'iou of class 6': val_iou[-1][6],
                                           'iou of class 7': val_iou[-1][7]}, epoch)
            
        print(epoch, train_losses[-1], mean_iou[-1])
        
    torch.save(model.state_dict(), "./SegNet_with_class_weight(median)_results/final_model.prm")

In [42]:
model = SegNetBasic(3, 8)
writer = SummaryWriter("./SegNet_with_class_weight(median)_results/")
train_model(model, train_loader, test_loader, criterion=nn.CrossEntropyLoss(weight=class_weight.to('cuda')), device="cuda", writer=writer)

  0%|          | 1/2310 [00:00<05:22,  7.16it/s]

0 0.9479384233407925 0.4458967447280884


  0%|          | 0/2310 [00:00<?, ?it/s]

1 0.31126673399898697 0.5815553069114685


  0%|          | 0/2310 [00:00<?, ?it/s]

...

197 0.025764800707784218 0.7745466232299805


  0%|          | 0/2310 [00:00<?, ?it/s]

198 0.023398797412960755 0.7772853374481201


100%|██████████| 2310/2310 [07:59<00:00,  4.31it/s]


199 0.025647739554287913 0.7147150039672852


In [16]:
colors = torch.tensor([[0, 0, 0],         # class 0 'background'  black
                       [255, 0, 0],       # class 1 'grasp'       red
                       [255, 255, 0],     # class 2 'cut'         yellow
                       [0, 255, 0],       # class 3 'scoop'       green
                       [0, 255, 255],     # class 4 'contain'     sky blue
                       [0, 0, 255],       # class 5 'pound'       blue
                       [255, 0, 255],     # class 6 'support'     purple
                       [255, 255, 255]    # class 7 'wrap grasp'  white
                      ])

In [17]:
def class_to_mask(cls):
    
    mask = colors[cls].transpose(1, 2).transpose(1, 3)
    
    return mask

In [18]:
def predict(model, sample, device='cpu'):
    model.eval()
    model.to(device)
    
    x, y = sample['image'], sample['class']
    
    x = x.to(device)
    y = y.to(device)

    with torch.no_grad():
        _, y_pred = model(x).max(1)    # y_pred.shape => (N, 240, 320)
    
    true_mask = class_to_mask(y).to('cpu')
    pred_mask = class_to_mask(y_pred).to('cpu')
    
    save_image(true_mask, "./SegNet_with_class_weight(median)_results/true_mask_with_SegNet_with_class_weight(median).jpg")
    save_image(pred_mask, "./SegNet_with_class_weight(median)_results/pred_mask_with_SegNet_with_class_weight(median).jpg")

In [40]:
trained_model = SegNetBasic(3, 8)
trained_model.load_state_dict(torch.load("./SegNet_with_class_weight(median)_results/best_iou_model.prm"))

In [41]:
eval_data = PartAffordanceDataset('eval.csv',
                                transform=transforms.Compose([
                                    CenterCrop(),
                                    ToTensor(),
                                    Normalize()
                                ]))

In [42]:
def reverse_normalize(x, mean, std):
    x[:, 0, :, :] = x[:, 0, :, :] * std[0] + mean[0]
    x[:, 1, :, :] = x[:, 1, :, :] * std[1] + mean[1]
    x[:, 2, :, :] = x[:, 2, :, :] * std[2] + mean[2]
    return x

In [43]:
eval_loader = DataLoader(eval_data, batch_size=8, shuffle=False)

In [46]:
mean=[55.8630, 59.9099, 91.7419]
std=[31.6852, 29.8496, 19.0835]

for sample in eval_loader:
    trained_model.eval()
    
    predict(trained_model, sample)
    
    x = sample["image"]
    x = reverse_normalize(x, mean, std)
    save_image(x/255, "./SegNet_with_class_weight(median)_results/original_img_with_SegNet_with_class_weight(median).jpg")
    
    break